In [1]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand

spark = SparkSession.builder.appName("simulator").master("local").config("spark.driver.memory", "40g") \
  .config("spark.executor.memory", "50g").getOrCreate()


24/05/16 22:37:55 WARN Utils: Your hostname, Celal-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.169 instead (on interface en0)
24/05/16 22:37:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 22:37:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/veriyogun/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/veriyogun/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/envs/veriyogun/lib/python3.9/socket.

KeyboardInterrupt: 

In [ ]:
import numpy as np

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
import json

with open('apikey.json') as f:
    data = json.load(f)
    api_key = data['key']
    client_id = data['client_id']
    user_agent = data['user_agent']

with open('mongo.json') as m:
    data = json.load(m)
    conn_string = data["connection_string"]

In [ ]:
df = spark.read.csv("turkish-subreddits.csv", header=True)
df.show()

+-----------------+
|       Subreddits|
+-----------------+
|        liseliler|
|        Hacettepe|
|          Yatirim|
|           kripto|
|           Finans|
|         Videoyun|
|  androidoyunclub|
|       burdurland|
|            Kafes|
|sariyerbelediyesi|
|      TurkeyJerky|
+-----------------+



In [ ]:
df_with_random = df.withColumn("random", rand())
random_row_1 = df_with_random.orderBy("random").limit(1)
random_row_1.head()["Subreddits"]

'TurkeyJerky'

In [ ]:
import praw

In [ ]:
# PRAW ile Reddit'e bağlan
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=api_key,
    user_agent=user_agent
)


In [ ]:
import requests as req
import time 
import random

import pymongo

In [ ]:
mongo_client = pymongo.MongoClient(conn_string)

mydb = mongo_client["bigData"]

In [ ]:
def preprocess_text(text):
    # Özel karakterleri kaldırma
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Küçük harfe dönüşüm
    text = text.lower()
    
    # Gereksiz boşlukları temizleme
    text = ' '.join(text.split())
    
    return text

In [ ]:


def remove_stopwords(text):
    # Metni tokenize et
    tokens = word_tokenize(text)
    
    # Stop word'leri yükle (İngilizce)
    stop_words = set(stopwords.words('turkish'))  # İngilizce stop word'leri kullanmak için
    
    # Stop word'leri metinden kaldır
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Temizlenmiş metni birleştir
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

In [ ]:
def Transformation(submission_data):
    # Eğer selftext değişkeni boşsa
    if not submission_data["selftext"]:
        # subreddit_name değişkenini selftext olarak kullan
        submission_data["selftext"] = submission_data["subreddit_name"]
    submission_data["selftext"] = remove_stopwords(preprocess_text(submission_data["selftext"]))
    submission_data["title"] = remove_stopwords(preprocess_text(submission_data["title"]))
    return submission_data

In [ ]:
def request_submission(submission_data):
    url = "http://192.168.0.159:8080/sentiment"  

    if not submission_data.get("selftext"):
        submission_data["selftext"] = submission_data["title"]

    body = {
        "text" : submission_data["selftext"]
    }

    
    # JSON verisini oluştur
    payload = json.dumps(body)
    
    # POST isteği gönder
    response = req.post(url, data=payload, headers={'Content-Type': 'application/json'})
    
    # Yanıtı kontrol et
    if response.status_code == 200:
        #print("Submission successfully posted!")
        response_data = response.json()  # Yanıtı JSON formatında al
        submission_data["sentiment"] = response_data.get("sentiment", -1)
    else:
        print(f"Failed to post submission. Status code: {response.status_code}, Response: {response.text}")

    
    return submission_data

In [ ]:
while True:
    df_with_random = df.withColumn("random", rand())
    random_row_1 = df_with_random.orderBy("random").limit(1)
    subreddit_name = random_row_1.head()["Subreddits"]
    subreddit = reddit.subreddit(subreddit_name)
    
    random_hot_limit = random.randint(1, 100)

    random_hot_limit = random.randint(1, 100)
    hot_submissions = reddit.subreddit(subreddit_name).hot(limit=random_hot_limit)

    # Choose a random submission from the fetched results
    random_submission = random.choice(list(hot_submissions))

    # Extract relevant information from the submission
    submission_data = {
        "subreddit_name": subreddit_name,
        "title": random_submission.title,
        "permalink": random_submission.permalink,
        "selftext": random_submission.selftext  # Add selftext if available
    }
    print("-----önce------")
    print(submission_data)
    submission_data = request_submission(submission_data)
    print("-------------sonra--------------------")
    print(submission_data)
    submission_data = 
    # veritabanına ekleme
    collection = mydb[submission_data["subreddit_name"]]
    x = collection.insert_one(submission_data)
    time.sleep(3)

-----önce------
{'subreddit_name': 'TurkeyJerky', 'title': '%100 orjinal meme', 'permalink': '/r/TurkeyJerky/comments/1cmibl1/100_orjinal_meme/', 'selftext': ''}
-------------sonra--------------------
{'subreddit_name': 'TurkeyJerky', 'title': '%100 orjinal meme', 'permalink': '/r/TurkeyJerky/comments/1cmibl1/100_orjinal_meme/', 'selftext': '%100 orjinal meme', 'sentiment': 1}
-----önce------
{'subreddit_name': 'Finans', 'title': 'İşsizlik maaşı nasıl alınır? işsizlik ödeneğinden yararlanma şartları ne...', 'permalink': '/r/Finans/comments/16uif9c/işsizlik_maaşı_nasıl_alınır_işsizlik_ödeneğinden/', 'selftext': ''}
-------------sonra--------------------
{'subreddit_name': 'Finans', 'title': 'İşsizlik maaşı nasıl alınır? işsizlik ödeneğinden yararlanma şartları ne...', 'permalink': '/r/Finans/comments/16uif9c/işsizlik_maaşı_nasıl_alınır_işsizlik_ödeneğinden/', 'selftext': 'İşsizlik maaşı nasıl alınır? işsizlik ödeneğinden yararlanma şartları ne...', 'sentiment': 1}
-----önce------
{'subr

KeyboardInterrupt: 

ETL Transformation işlemi